In [1]:
import os

In [2]:
os.listdir('/home/udit/programs/LTHT/data/saves/fc1/mnist/0/')

['5_model_lt_20_5.pth.tar',
 '8_model_lt_20_27.pth.tar',
 '8_model_lt_20_12.pth.tar',
 '0_model_lt_20_15.pth.tar',
 '4_model_lt_20_16.pth.tar',
 '1_model_lt_20_19.pth.tar',
 '9_model_lt_20_3.pth.tar',
 '9_model_lt_20_30.pth.tar',
 '7_model_lt_20_15.pth.tar',
 '1_model_lt_20_1.pth.tar',
 '8_model_lt_20_6.pth.tar',
 '1_model_lt_20_27.pth.tar',
 '2_model_lt_20_23.pth.tar',
 '7_model_lt_20_1.pth.tar',
 '3_model_lt_20_16.pth.tar',
 '0_model_lt_20_26.pth.tar',
 '5_model_lt_20_6.pth.tar',
 '1_model_lt_20_4.pth.tar',
 '5_model_lt_20_1.pth.tar',
 '6_model_lt_20_0.pth.tar',
 '1_model_lt_20_0.pth.tar',
 '2_model_lt_20_4.pth.tar',
 '7_model_lt_20_8.pth.tar',
 '4_model_lt_20_3.pth.tar',
 '7_model_lt_20_4.pth.tar',
 '5_model_lt_20_19.pth.tar',
 '0_model_lt_20_2.pth.tar',
 '0_model_lt_20_7.pth.tar',
 '7_model_lt_20_0.pth.tar',
 '5_model_lt_20_0.pth.tar',
 '3_model_lt_20_2.pth.tar',
 '9_model_lt_20_18.pth.tar',
 '8_model_lt_20_1.pth.tar',
 '5_model_lt_20_25.pth.tar',
 '5_model_lt_20_2.pth.tar',
 '2_mo

In [3]:
!pip install Cython 

In [4]:
!pip install persim Ripser networkx

In [5]:
ROOT_DIR = '/home/udit/programs/LTHT/data/saves/' # need absolute root dir location
model_list = ['fc1', 'resnet18', 'vgg16']
dataset_list = ['mnist', 'cifar10']
random_seed = ['0', '42', '1337']
#msk_pct = 90 # prune percentage for LT network
#model_loc0 = 'saves/{}/{}/{}/0_model_lt.pth.tar'.format(model_name, dataset_name, random_seed) # location of saved, un-pruned model 
#model_loc1 = 'saves/{}/{}/4_model_lt.pth.tar'.format(model_name, dataset_name) # location of saved, pruned model (after 1 prune iteration)

In [6]:
def get_best_training_file_for_pruning_iteration(pruning_iteration, trained_files):
    best_epoch = 0
    best_file = ""
    for file in trained_files:
        if (file.startswith(str(pruning_iteration)+"_")):
            file_name = file.split('.')[0]
            curr_epoch = int(file_name.split('_')[-1])
            if (best_epoch < curr_epoch):
                best_epoch = curr_epoch
                best_file = file
    return best_file

def get_best_training_files_in_dir(curr_dir):
    best_model_per_pruning_it_list = []
    model_trained_files = sorted(os.listdir(curr_dir))
    pruning_iterations_done = []
    for file in model_trained_files:
        curr_pruning_iteration = file.split('_')[0]
        if curr_pruning_iteration not in pruning_iterations_done:
            best_training_file = get_best_training_file_for_pruning_iteration(curr_pruning_iteration, model_trained_files)
            if (len(best_training_file) != 0):
                best_model_per_pruning_it_list.append(best_training_file)
            pruning_iterations_done.append(curr_pruning_iteration)
    return best_model_per_pruning_it_list

In [8]:
for model in model_list:
    for dataset in dataset_list:
        for seed in random_seed:
            model_dataset_seed_dir = ROOT_DIR + "{}/{}/{}/".format(model, dataset, seed)
            if (os.path.isdir(model_dataset_seed_dir)):
                print(model_dataset_seed_dir)
                best_model_per_pruning_it_list = get_best_training_files_in_dir(model_dataset_seed_dir)
                compute_homology(model_dataset_seed_dir, best_model_per_pruning_it_list)
            

/home/udit/programs/LTHT/data/saves/fc1/mnist/0/
/home/udit/programs/LTHT/data/saves/fc1/mnist/42/
/home/udit/programs/LTHT/data/saves/fc1/mnist/1337/
/home/udit/programs/LTHT/data/saves/resnet18/cifar10/0/
/home/udit/programs/LTHT/data/saves/resnet18/cifar10/42/
/home/udit/programs/LTHT/data/saves/resnet18/cifar10/1337/
/home/udit/programs/LTHT/data/saves/vgg16/cifar10/0/
/home/udit/programs/LTHT/data/saves/vgg16/cifar10/42/
/home/udit/programs/LTHT/data/saves/vgg16/cifar10/1337/


In [7]:
# Compute homology per model-dataset-seed
def compute_homology(root_dir, file_list):
    pass


In [9]:
import networkx as nx
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import numpy as np
import scipy.sparse
import matplotlib.pyplot as plt

import persim # see persim.scikit-tda.org
from ripser import ripser # see ripser.scikit-tda.org

In [10]:
def test(model, test_loader, criterion):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        accuracy = 100. * correct / len(test_loader.dataset)
    return accuracy


# primary function for translating NN to networkx graph. 
def to_directed_networkx(params, layer_names=None):
    '''primary function for translating FCNN to networkx graph. 
    Gives node a name corresponding to its layer and location in 
    parameter matrix. Weights edges to match notion of "distance"
    matrix for nodes. 
    
    params (list): list of FC weight matrices in numpy format.
    layer_names (optional): list of layer names to be associated to each node
    (defaults to integer layer number).
    
    returns: networkx DiGraph.
    '''
    if layer_names is None:
        layer_names = [i for i in range(len(params))]
    G = nx.DiGraph()
    for i in range(len(params)):
        layer_name = layer_names[i]
        p = scipy.sparse.coo_matrix(params[i].T)
        for i,j,v in zip(p.row, p.col, p.data):
            if v != 0:
                G.add_edge(str(layer_name)+'_'+str(i),str(layer_name+1)+'_'+str(j),weight=np.abs(1/1+v))
    return G

In [11]:
# import torch
model = torch.load('/home/udit/programs/LTHT/data/saves/fc1/mnist/0/0_model_lt_20_47.pth.tar')
# params = []
# for name, param in model.named_parameters():
#     if 'weight' in name:
#         pnum = param.data.cpu().numpy()
#         print('Parameter Shape: ', pnum.shape)
#         params.append(pnum)
        
# G = to_directed_networkx(params)

ModuleNotFoundError: No module named 'archs'